In [2]:
import pandas as pd

filename = 'gd1_candidates.hdf5'
candidate_df = pd.read_hdf(filename, 'candidate_df')

In [4]:
from astropy.table import Table

candidate_table = Table.from_pandas(candidate_df)
type(candidate_table)

astropy.table.table.Table

In [5]:
table = candidate_table[['source_id']]
table.write('candidate_df.xml', format='votable', overwrite=True)

In [17]:
query = """SELECT *
FROM gaiadr2.panstarrs1_best_neighbour as best
JOIN tap_upload.candidate_df as candidate_df
ON best.source_id = candidate_df.source_id
"""

In [18]:
from astroquery.gaia import Gaia

job = Gaia.launch_job_async(query=query, 
                            upload_resource='candidate_df.xml', 
                            upload_table_name='candidate_df')

INFO: Query finished. [astroquery.utils.tap.core]


In [19]:
results = job.get_results()
results

source_id,original_ext_source_id,angular_distance,number_of_neighbours,number_of_mates,best_neighbour_multiplicity,gaia_astrometric_params,source_id_2
,,arcsec,,,,,
int64,int64,float64,int32,int16,int16,int16,int64
635860218726658176,130911385187671349,0.053667035895467084,1,0,1,5,635860218726658176
635674126383965568,130831388428488720,0.038810268141577516,1,0,1,5,635674126383965568
635535454774983040,130631378377657369,0.034323028828991076,1,0,1,5,635535454774983040
635497276810313600,130811380445631930,0.04720255413250006,1,0,1,5,635497276810313600
635614168640132864,130571395922140135,0.020304189709964143,1,0,1,5,635614168640132864
635598607974369792,130341392091279513,0.036524626853403054,1,0,1,5,635598607974369792
635737661835496576,131001399333502136,0.036626827820716606,1,0,1,5,635737661835496576
635850945892748672,132011398654934147,0.021178742393378396,1,0,1,5,635850945892748672


In [21]:
table_ext = results[['source_id', 'original_ext_source_id']]
table_ext.write('external.xml', format='votable', overwrite=True)

In [22]:
query2 = """SELECT
external.source_id, ps.g_mean_psf_mag, ps.i_mean_psf_mag
FROM gaiadr2.panstarrs1_original_valid as ps
JOIN tap_upload.external as external
ON ps.obj_id = external.original_ext_source_id
"""

In [23]:
job2 = Gaia.launch_job_async(query=query2, 
                       upload_resource='external.xml', 
                       upload_table_name='external')

INFO: Query finished. [astroquery.utils.tap.core]


In [24]:
results2 = job2.get_results()
results2

source_id,g_mean_psf_mag,i_mean_psf_mag
,,mag
int64,float64,float64
635860218726658176,17.8978004455566,17.5174007415771
635674126383965568,19.2873001098633,17.6781005859375
635535454774983040,16.9237995147705,16.478099822998
635497276810313600,19.9242000579834,18.3339996337891
635614168640132864,16.1515998840332,14.6662998199463
635598607974369792,16.5223999023438,16.1375007629395
635737661835496576,14.5032997131348,13.9849004745483
635850945892748672,16.5174999237061,16.0450000762939


In [25]:
filename = 'gd1_photo.fits'
results2.write(filename, overwrite=True)